In [1]:
# load the dataset

from tensorflow.keras.datasets import mnist
#from progressbar import ProgressBar as progressbar
from progressbar import *
import numpy as np
import multiprocessing as mp
from itertools import product
import sys

from layers.Conv2D import Conv2D
from layers.Pooling import Pooling
from layers.Dense import Dense
from layers.Flatten import Flatten

2022-10-16 06:28:53.316300: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# import the data
(train_X, train_y), (test_X, test_y) = mnist.load_data()

# scale the data
train_X, test_X = train_X / 255.0, test_X / 255.0


# reduce the size of the dataset
train_X, test_X = train_X[:1000], test_X[:1000]
train_y, test_y = train_y[:1000], test_y[:1000]

# need the fourth dimension to represent the number of channels
train_X = train_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)

print('Train: X=%s, y=%s' % (train_X.shape, train_y.shape))
print('Test: X=%s, y=%s' % (test_X.shape, test_y.shape))

Train: X=(1000, 28, 28, 1), y=(1000,)
Test: X=(1000, 28, 28, 1), y=(1000,)


In [3]:
# define the model
model = []

model.append(Conv2D(32, 2, 1, 1))
model.append(Pooling(2, 2, 'max'))
model.append(Flatten())

# determine the number of input features by running one forward pass on one image
dims = train_X[0]
dims = dims.reshape(1, *dims.shape)
for layer in model:
    dims = layer.forward(dims)


model.append(Dense(np.prod(dims.shape[1]), 10))
#model.append(Dense(np.prod(dims.shape[1]), 128))
#model.append(Dense(128, 10))

for layer in model[3:]:
    dims = layer.forward(dims)

In [4]:
import matplotlib.pyplot as plt

def plot_image(image):
    plt.imshow(image, cmap='gray')
    plt.show()

In [5]:
def predict(X, model):
    # forward pass on a single image
    #plot_image(X[0])            
    for layer in model:
        X = layer.forward(X)

        #print('layer %s, X %s' % (layer.name, X.shape))
        #plot_image(X[0, :, :, 0, 0])            

    return X


def train(X, y, model, lr=0.01, epochs=10):

    # need to make epochs work
    # need to do forward passes chunks of mp.cpu_count() images at a time
    # when each forward pass is done, do a backward pass on the same chunk of images
    chunksize = 100
    # create a one-hot vector of y
    #y = np.eye(10)[y]
    loss = 0
    accuracy = 0
    widgets = [FormatLabel(''), ' Training: ', Percentage(), ' ', SimpleProgress(), ' ', Bar(), ' ', ETA(), ' ', FormatLabel(''), ' ', FormatLabel('')]
    for epoch in range(epochs):
        widgets[0] = FormatLabel('(epoch {:.0f})'.format(epoch))
        p = ProgressBar(max_value=X.shape[0], widgets=widgets, redirect_stdout=True)
        # break into chunks of at most mp.cpu_count() images
        for i in range(0, len(X), chunksize):
            loss = 0
            accuracy = 0
            # forward pass
            y_pred = predict(X[i:min(X.shape[0], i+chunksize)], model)

            
            # gradient
            #cross_entropy loss
            for j in range(chunksize):
                for d in range(y_pred.shape[2]):
                    loss += -np.log(y_pred[j, y[j+i], d])
                    accuracy += 1 if np.argmax(y_pred[j, :, d]) == y[j+i] else 0
            loss /= chunksize
            accuracy /= chunksize * 100
            widgets[10] = FormatLabel('(loss: {:.16f}, '.format(loss))
            widgets[12] = FormatLabel('accuracy: {:.4f})'.format(accuracy))

            grad = np.zeros(y_pred.shape)
            for j in range(chunksize):
                grad[y[j+i]] = -1 / y_pred[j, y[j+i]]

            # backward pass
            for layer in reversed(model):
                grad = layer.backward(grad, lr)
            
            p.update(i)
            sys.stdout.flush()

        p.finish()


In [6]:
# train the model
train(train_X, train_y, model)

IndexError: index 5 is out of bounds for axis 0 with size 1